<header style="background-color: black; color: white; padding: 10px 20px 10px 20px;">
    <h1>My Portfolio Management System</h1>
    <p>This notebook tracks investment portfolios for client managers</p>
</header>

In [10]:
from datetime import datetime as dt
import uuid
import unittest
import pandas as pd
#import pandas_datareader as pdr
import numpy as np
import mysql.connector
from IPython.core.display import HTML
import ipywidgets as widgets
import time
import requests
from bs4 import BeautifulSoup
import json


#db password: *******
#CREATE TABLE `mis2100example_portfolio_db`.`Portfolio` ( pid INT NOT NULL AUTO_INCREMENT , name VARCHAR(255) NOT NULL , cid INT NOT NULL , PRIMARY KEY (pid), CONSTRAINT FOREIGN KEY pidcid (cid) REFERENCES Client(cid)) ENGINE = MyISAM;
#CREATE TABLE `mis2100example_portfolio_db`.`Stock` ( sid INT NOT NULL AUTO_INCREMENT , ticker VARCHAR(10) NOT NULL , name VARCHAR(255) NOT NULL , PRIMARY KEY (sid)) ENGINE = MyISAM;
#CREATE TABLE `mis2100example_portfolio_db`.`PortfolioStockPurchase` ( sid INT NOT NULL , pid INT NOT NULL, datetime DATETIME NOT NULL, price DECIMAL(5,2) NOT NULL, quantity INT NOT NULL, PRIMARY KEY (pid, sid, datetime), CONSTRAINT FOREIGN KEY pspidpid (pid) REFERENCES Portfolio(pid), CONSTRAINT FOREIGN KEY pspidsid (sid) REFERENCES Stock(sid)) ENGINE = MyISAM;
#CREATE TABLE `mis2100example_portfolio_db`.`ClientManager` ( cmid INT NOT NULL AUTO_INCREMENT, name VARCHAR(255) NOT NULL, PRIMARY KEY (cmid)) ENGINE = MyISAM;

In [11]:
%%html
<style>
section {
    border: solid black 1px;
    margin: 5px 0px 5px 0px;
}

.sectionHeader {
    background-color: black;
    color: white;
    font-size: 18px;
    padding: 10px;
}

.sectionMain {
    padding: 0px;
}

.sectionFooter {
    background-color: #009933;
    color: white;
    font-size: 16px;
    padding: 10px;
    border-top: 1px solid black;
}

.lightRow {
    background-color: #FFF;
    padding: 10px;
    margin: 0px;
}

.lightRow:hover {
    background-color: #333;
    color: white;
}

.darkRow {
    background-color: #EEE;
    padding: 10px;
    margin: 0px;
}

.darkRow:hover {
    background-color: #333;
    color: white;
}

.warningMessage {
    background-color: #CC0000;
    color: white;
    padding: 10px;
}

.successMessage {
    background-color: #009933;
    color: white;
    padding: 10px;
}
</style>

In [12]:
#Assignment 8
class UnsupportedStockException(Exception):
    pass

class UnselectedPortfolioException(Exception):
    pass

In [13]:
#Assignment 2
class Client:
    def __init__(self, dbConnectorP):
        self.dbConnector = dbConnectorP #Assignment 7
        
        self.clientId = None
        self.firstName = None
        self.lastName = None
        self.phone = None
    
    #Assignment 7
    def create(self, clientIdP, firstNameP, lastNameP, phoneP):
        statementString = "INSERT INTO Client SET clientId=?, cmid=1, firstName=?, lastName=?, phone=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (clientIdP, firstNameP, lastNameP, phoneP))
    
        if self.dbConnector.statement.rowcount > 0:
            self.clientId = clientIdP
            self.firstName = firstNameP
            self.lastName = lastNameP
            self.phone = phoneP
            
        self.dbConnector.clearStatement()
    
    #Assignment 7
    def findByClientId(self, clientIdP):
        statementString = "SELECT clientId, firstName, lastName, phone FROM Client WHERE clientId=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (clientIdP, ))
         
        result = self.dbConnector.statement.fetchone()
        
        if result:
            (clientId, firstName, lastName, phone) = result
            
            self.clientId = clientId
            self.firstName = firstName
            self.lastName = lastName
            self.phone = phone
                
        self.dbConnector.clearStatement()
        
        return result
    
    #Assignment 7
    def findByLastName(self, lastNameP):
        statementString = "SELECT clientId, firstName, lastName, phone FROM Client WHERE lastName=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (lastNameP, ))
        
        result = self.dbConnector.statement.fetchall()
        
        self.dbConnector.clearStatement()
        
        return result
    
    #Assignment 7
    def updatePhone(self, clientIdP, phoneP):
        statementString = "UPDATE Client SET phone=? WHERE clientId=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (phoneP, clientIdP))
        
        if self.dbConnector.statement.rowcount > 0:
            return True
        
        return False

######################################################################################################    
    
class Portfolio:
    def __init__(self, dbConnectorP):
        self.dbConnector = dbConnectorP #Assignment 7
        
        self.name = None
        self.client = None
        self.portfolioId = None #Assignment 7
           
    #Assignment 7
    def create(self, nameP, clientObj):
        statementString = "INSERT INTO Portfolio SET name=?, cid=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (nameP, clientObj.clientId))
        
        if self.dbConnector.statement.rowcount > 0:
            self.portfolioId = self.dbConnector.statement.lastrowid
            self.name = nameP
            self.client = clientObj
            
        self.dbConnector.clearStatement()

    #Assignment 7
    def findByPortfolioId(self, portfolioId):
        statementString = "SELECT pid as portfolioId, name FROM Portfolio WHERE pid=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (portfolioId, ))
        
        result = self.dbConnector.statement.fetchone()
        
        if result:
            (portfolioId, name) = result
            
            self.portfolioId = portfolioId
            self.name = name
                
        self.dbConnector.clearStatement()
    
    #Assignment 7
    def findByClientId(self, clientId):
        statementString = "SELECT pid as portfolioId, name FROM Portfolio WHERE cid=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (clientId, ))
        
        result = self.dbConnector.statement.fetchall()
        
        self.dbConnector.clearStatement()
        
        return result
    
    #Assignment 7
    def updateName(self, portfolioIdP, nameP):
        statementString = "UPDATE Portfolio SET name=? WHERE portfolioId=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (nameP, portfolioId))
        
        if self.dbConnector.statement.rowcount > 0:
            return True
        
        return False
    
    #Assignment 7
    def purchaseStock(self, ticker, price, quantity):
        purchase = StockPurchase(self.dbConnector, self)
        return purchase.execute(ticker, price, quantity)
    
    #Assignment 7
    def listPurchases(self):
        statementString = "SELECT sp.sid as tickerId, s.ticker as ticker, sp.price as price, sp.quantity as quantity, sp.datetime as purchaseDate FROM PortfolioStockPurchase as sp INNER JOIN Stock as s ON(sp.sid=s.sid) WHERE sp.pid=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (self.portfolioId, ))
        
        result = self.dbConnector.statement.fetchall()
        
        self.dbConnector.clearStatement() 
        
        return result

######################################################################################################
    
class Stock:
    def __init__(self, dbConnectorP):
        self.dbConnector = dbConnectorP
        self.ticker = None
        self.tickerId = None
        self.history = None #Assignment 6
    
    #Assignment 7
    def setTicker(self, tickerP):
        statementString = "SELECT sid as tickerId, ticker FROM Stock WHERE ticker=?"
        
        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (tickerP, ))
        
        result = self.dbConnector.statement.fetchone()
        
        if self.dbConnector.statement.rowcount > 0:
            (tickerId, ticker) = result
            
            self.ticker = ticker
            self.tickerId = tickerId
            self.dbConnector.clearStatement() 
            return True
        else:
            self.dbConnector.clearStatement() 
            return False               
    
    #Assignment 7
    def listAvailableStocks(self):
        statementString = "SELECT sid as tickerId, ticker FROM Stock"
        
        self.dbConnector.createStatement()
        self.dbConnector.executeStatement(statementString)
        
        result = self.dbConnector.statement.fetchall()
        
        self.dbConnector.clearStatement()
        
        return result
        
    def getStockHistory(self, startDate, endDate):
        av = AlphaVantageConnector("4U3NNSG5OHR1CBIG")
        av.getDaily(self.ticker)
        data = av.toDataFrame()
        
        self.history = data[(data.index >= startDate) & (data.index <= endDate)]
        return self.history

######################################################################################################    

class StockPurchase:
    def __init__(self, dbConnectorP, portfolioObj=None):
        self.dbConnector = dbConnectorP        
        self.portfolio = portfolioObj
    
    def execute(self, ticker, price, quantity):
        #Assignment 7
        stock = Stock(self.dbConnector)
        
        if not stock.setTicker(ticker):
            raise UnsupportedStockException("The stock ticker you requested is misspelled or is not supported by the system.")
        
        if not isinstance(self.portfolio, Portfolio) or self.portfolio.portfolioId is None:
            raise UnselectedPortfolioException("You cannot make a purchase until a portfolio has been selected.")
        
        statementString = "INSERT INTO PortfolioStockPurchase SET sid=?, pid=?, datetime=now(), price=?, quantity=?"

        self.dbConnector.createStatement(prepared=True)
        self.dbConnector.executeStatement(statementString, (stock.tickerId, self.portfolio.portfolioId, price, quantity))
        
        if self.dbConnector.statement.rowcount > 0:
            self.dbConnector.clearStatement()
            return True
        else:
            self.dbConnector.clearStatement()
            return False        

######################################################################################################        
                
#Assignment 7
#creation of the database driver interface
class DatabaseDriverInterface:
    def __init__(self, hostP, databaseP, userP, passwordP):
        pass

    def connect(self):
        pass

    def disconnect(self):
        pass

    def createStatement(self, prepared=False):
        pass

    def executeStatement(self, statementString, data={}):
        pass

    def clearStatement(self):
        pass

    def commit(self):
        pass

    def raiseException(self, message):
        pass

#Assignment 7
#implementation of the database driver interface
class MySQLConnector(DatabaseDriverInterface):
    def __init__(self, hostP, databaseP, userP, passwordP):
        self.dbConnection = None
        self.host = hostP
        self.database = databaseP
        self.user = userP
        self.password = passwordP
        self.statement = None

    def connect(self):
        try:
            self.dbConnection = mysql.connector.connect(
                user=self.user, 
                password=self.password,
                host=self.host,
                database=self.database )
        except mysql.connector.Error as err:
            print(err)
    
    def disconnect(self):
        self.dbConnection.close()

    def createStatement(self, prepared=False):
        if not prepared: 
            self.statement = self.dbConnection.cursor()
        else:
            self.statement = self.dbConnection.cursor(prepared=True)

    def executeStatement(self, statementString, data=None):
        try:           
            if isinstance(self.statement, mysql.connector.cursor_cext.CMySQLCursorPrepared):
                self.statement.execute(statementString, data)
            else:
                self.statement.execute(statementString)
        except mysql.connector.Error as err:
            print(err)
            
    def clearStatement(self):
        self.statement.close()
        
    def commit(self):
        try:
            self.dbConnection.commit()
        except mysql.connector.Error as err:
            print(err)

    def raiseException(self, message):
        raise mysql.connector.Error(message)   

In [14]:
######################################################################################################        

#Assignment 3
class ClientManager:
    def __init__(self, dbConnectorP):
        self.currentClient = None
        self.currentPortfolio = None #Assignment 4
        self.dbConnector = dbConnectorP #Assignment 7
        
    def setView(self, viewP):
        self.view = viewP
    
    def createClient(self, firstNameP, lastNameP, phoneP):
        uniqueId = uuid.uuid4().hex
        client = Client(self.dbConnector)
        client.create(uniqueId, firstNameP, lastNameP, phoneP) #Assignment 7
        
        if client.clientId == uniqueId:
            self.currentClient = client
            html = f"<div class='successMessage'>Successfully created the client {self.currentClient.firstName} {self.currentClient.lastName} with Id {self.currentClient.clientId}</div>"
            display(HTML(html))
        else:
            html = "<div class='warningMessage'>Could not create the client at this time</div>"
            display(HTML(html))
    
    #Assignment 4
    def selectClientById(self, buttonWidget): #Assignment 4
        client = Client(self.dbConnector) #Assignment 7
        client.findByClientId(buttonWidget.clientId)
        
        if client.clientId is not None:
            self.currentClient = client
            portfolios = self.getClientPortfolios()
            self.view.buildPortfolioListResults(portfolios)
            self.view.showSuccessMessage(f"You are now viewing {self.currentClient.firstName} {self.currentClient.lastName}'s account")
        else:
            self.view.showWarningMessage("The client id that you requested does not exist. Please make sure the client id is valid")
            
    #Assignment 7
    def findByLastName(self, buttonWidget):
        client = Client(self.dbConnector)
        clients = client.findByLastName(self.view.clientSearchBox.value)
        
        self.view.buildClientSearchResults(clients) #Chapter 9
        
    def createPortfolio(self, portfolioNameP):
        #Assignment 7
        portfolio = Portfolio(self.dbConnector)
        portfolio.create(portfolioNameP, self.currentClient)
            
        if portfolio.portfolioId is not None:
            self.currentPortfolio = portfolio
            html = f"<div class='successMessage'>Successfully created the portfolio {self.currentPortfolio.name} for {self.currentClient.firstName} {self.currentClient.lastName}</div>"
            display(HTML(html))
        else:
            html = "<div class='warningMessage'>Could not create the portfolio at this time</div>"
            display(HTML(html))
    
    #Assignment 4
    def selectPortfolioById(self, buttonWidget):
        #Assignment 7
        portfolio = Portfolio(self.dbConnector)
        portfolio.findByPortfolioId(buttonWidget.portfolioId)
        
        if portfolio.portfolioId is not None:
            self.currentPortfolio = portfolio
            self.view.showSuccessMessage(f"Changed to portfolio {self.currentPortfolio.name} for {self.currentClient.firstName} {self.currentClient.lastName}")
        else:
            self.view.showWarningMessage("Could not change to the requested portfolio at this time")
    
    #Assignment 7
    def listAvailableStocks(self):
        stock = Stock(self.dbConnector)
        stocks = stock.listAvailableStocks()
        
        headerText = "The following stocks are available for purchase:"  #Assignment 8
        resultsText = ""  #Assignment 8
        
        counter = 0
        for (tickerId, ticker) in stocks:
            rowClassName = "darkRow"
            
            if counter % 2 == 0:
                rowClassName = "lightRow"
            
            resultsText += f"<div class='{rowClassName}'>{ticker} with id {tickerId}</div>"
            counter += 1
        
        #Assignment 8
        html = f"""
                <section>
                    <header class='sectionHeader'>{headerText}</header>
                    <main class='sectionMain'>{resultsText}</main>
                </section>
            """
        
        display(HTML(html))    
    
    #Assignment 7
    def purchaseStock(self, ticker, price, quantity):
        if not isinstance(self.currentPortfolio, Portfolio) or self.currentPortfolio.portfolioId is None:
            html = "<div class='warningMessage'>You must select a portfolio to purchase stock</div>"
            display(HTML(html))    
        else:
            try:
                if self.currentPortfolio.purchaseStock(ticker, price, quantity):
                    html = f"<div class='successMessage'>Your purchase for {quantity} shares of {ticker} at ${price} was successful at {dt.now()}</div>"
                    display(HTML(html))
            except Exception as e:
                html = f"<div class='warningMessage'>{e}</div>"
                display(HTML(html))            
    
    #Assignment 7
    def getClientPortfolios(self):
        portfolio = Portfolio(self.dbConnector)
        portfolios = portfolio.findByClientId(self.currentClient.clientId)

        return portfolios
    
    #Assignment 7
    def displayPortfolioPurchases(self):
        if not isinstance(self.currentPortfolio, Portfolio) or self.currentPortfolio.portfolioId is None:
            print("You must select a portfolio to list stock purchases")
        else:
            purchases = self.currentPortfolio.listPurchases()
            
            headerText = f"The following purchases have been made for the portfolio {self.currentPortfolio.name}:"  #Assignment 8
            resultsText = ""  #Assignment 8
            
            totalPurchasePrice = 0
            
            counter = 0
            for (tickerId, ticker, price, quantity, purchaseDate) in purchases:
                rowClassName = "darkRow"
            
                if counter % 2 == 0:
                    rowClassName = "lightRow"
                
                totalPurchasePrice += price*quantity
                
                resultsText += f"<div class='{rowClassName}'>Purchase {tickerId} for {ticker} was executed at {purchaseDate} for {quantity} shares at {price}</div>"
                counter += 1
            
            #Assignment 8
            html = f"""
                <section>
                    <header class='sectionHeader'>{headerText}</header>
                    <main class='sectionMain'>{resultsText}</main>
                    <footer class='sectionFooter'>Total Purchase Price: ${totalPurchasePrice}</footer>
                </section>
            """
        
            display(HTML(html))
    
    #Assignment 10
    def findCIKByCompanyName(self, buttonWidget):
        companyName = self.view.CIKSearchBox.value
        
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        response = requests.get(f"https://www.sec.gov/cgi-bin/cik_lookup?company={companyName}", headers=header)
        
        if response.ok:
            parser = BeautifulSoup(response.content, "html.parser")
            
            print(parser.find_all("pre"))
            
            if len(parser.find_all("pre")) > 1:
                CIKNumbers = parser.find_all("pre")[1]
                print(CIKNumbers.text)
                self.view.showMatchingCIKNumbers(CIKNumbers.text)
            else:
                self.view.showWarningMessage("The search returned 0 matching results. Please try another name")
        else:
            self.view.showWarningMessage(f"The CIK numbers could not be retrieved from the EDGAR website. The following error was received: {response.status_code} {response.reason}")
    
    #Assignment 10
    def getReportByCIK(self, buttonWidget):
        CIKNumber = self.view.CIKSelectBox.value
        
        header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

        #call the SEC API endpoint with the Oracle CIK number
        response = requests.get(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{CIKNumber}.json", headers=header)
        
        if response.ok:
            data = json.loads(response.content)
            
            #get the current liablities
            currentLiabilityReports = {}

            for liabilities in data["facts"]["us-gaap"]["LiabilitiesCurrent"]["units"]["USD"]:
                currentLiabilityReports[liabilities["end"]] = liabilities
            
            liabilitiesDF = pd.DataFrame(columns=["PeriodEnding","FiscalYear","FiscalPeriod","Type","CurrentLiabilities"])

            for liability in currentLiabilityReports.values():
                series = pd.Series({"PeriodEnding":liability["end"],"FiscalYear":liability["fy"],"FiscalPeriod":liability["fp"],"Type":liability["form"],"CurrentLiabilities":liability["val"]})
                liabilitiesDF = pd.concat([liabilitiesDF, series.to_frame().T])
            
            liabilitiesDF.set_index("PeriodEnding", inplace=True)

            #get the current assets
            currentAssetsReports = {}

            for assets in data["facts"]["us-gaap"]["AssetsCurrent"]["units"]["USD"]:
                currentAssetsReports[assets["end"]] = assets

            assetsDF = pd.DataFrame(columns=["PeriodEnding","FiscalYear","FiscalPeriod","Type","CurrentAssets"])

            for asset in currentAssetsReports.values():
                series = pd.Series({"PeriodEnding":asset["end"],"FiscalYear":asset["fy"],"FiscalPeriod":asset["fp"],"Type":asset["form"],"CurrentAssets":asset["val"]})
                assetsDF = pd.concat([assetsDF, series.to_frame().T])

            assetsDF.set_index("PeriodEnding", inplace=True)
            
            #merge the data and create the WorkingCapital and CurrentRatio
            df = pd.merge(liabilitiesDF, assetsDF, on="PeriodEnding", suffixes=('', '_y'))
            df = df.drop(columns=["FiscalYear_y","FiscalPeriod_y","Type_y"])
            df["WorkingCapital"] = df["CurrentAssets"] - df["CurrentLiabilities"]
            df["CurrentRatio"] = df["CurrentAssets"] / df["CurrentLiabilities"]
            
            #return the data frame back to the ClientManagerView to display it
            self.view.showCIKReport(df)
        else:
            self.view.showWarningMessage(f"A report could not be generated for the CIK number you entered. The following error was received: {response.status_code} {response.reason}")

######################################################################################################        
        
#Assignment 6
#simple class to get stock data from AlphaVantage API
class AlphaVantageConnector:
    def __init__(self, APIKey, dataType="json"):
        self.APIKey = APIKey
        self.dataType = dataType
        self.data = None
        
    def getDaily(self, ticker,  outputSize="full"):
        self.frequency = "daily"
        
        if self.dataType != "json":
            self.dataType = "csv"
        
        if outputSize != "full":
            outputSize = "compact"
        
        response = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&datatype={self.dataType}&outputsize={outputSize}&apikey={self.APIKey}")
        
        if response.ok:
            self.data = response.content
            return self.data
        else:
            raise Exception(f"{response.status_code}: {response.reason}")
    
    def toDictionary(self):
        if self.data is None:
            raise Exception("Cannot create dictionary. No data was retrieved from the API.")
        
        if self.dataType == "json":
            if self.frequency == "daily":
                dictionary = json.loads(self.data)
                return dictionary['Time Series (Daily)']
            
    def toDataFrame(self):
        if self.data is None:
            raise Exception("Cannot create data frame. No data was retrieved from the API.")
        
        if self.dataType == "json":
            df = pd.DataFrame.from_dict(self.toDictionary()).transpose().rename(
                columns={'1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'})
            df.index.name = "Date"
            return df
        
#Assignment 6
class StockDataCleaner:
    def __init__(self, dataP):
        self.data = dataP
        
        if self.data.index.name != "Date":
            self.data.set_index("Date")
            
        self.numericDataColumns = ["High","Low","Open","Close","Volume"]
    
    def isFloat(self, value):
        if value is None: 
            return False
        try:
            float(value)
            return True
        except:
            return False
    
    def forceNumericDataTypes(self):
        for column in self.numericDataColumns:
            if str(self.data.dtypes[column]) != "float64":
                print(f"The column {column} was not formatted with the correct data type. Trying to resolve the issue...")
                
                try:
                    self.data = self.data.astype({column:'float'})
                    print(f"Successfully converted the column {column} to a float data type")
                except Exception as e:
                    self.data[column] = self.data[column].apply(lambda value: np.nan if not self.isFloat(value) else value)
                    self.data = self.data.astype({column:'float'})
                    print(f"A data type issue was resolved for the column {column} by removing string data from the column")
            else:
                print(f"The column {column} was assigned the correct data type")
                
        display(self.data.dtypes)
                    
    def identifyOutliers(self):
        print("Outlier analysis (based on z-scores greater than 3 or less than -3)")
        
        for column in self.numericDataColumns:
            self.data[f"{column}ZScore"] = (self.data[column] - self.data[column].mean()) / self.data[column].std()
            
            outliers = self.data[(self.data[f"{column}ZScore"]>3) | (self.data[f"{column}ZScore"]<-3)]
            
            if outliers.empty:
                print(f"The column {column} had no clear outliers")
            else:
                print(f"The column {column} had the following outliers:")
                display(outliers)
                            
        self.data = self.data[self.numericDataColumns]

    def calculateSkewnessAndKurtosis(self):
        print("Skewness:")
        print(self.data[self.numericDataColumns].skew())
        print("Kurtosis:")
        print(self.data[self.numericDataColumns].kurtosis())
        
    def showHistogramByColumn(self, column):
        print(f"Histogram for {column}")
        histogram = self.data[column].hist(bins=20, figsize=(10,8))
        histogram.ticklabel_format(useOffset=False, style='plain')
        
    def imputeMissingData(self):
        for column in self.numericDataColumns:
            if column == "Volume":
                self.data[column] = self.data[column].fillna(self.data[column].median())
            else:
                self.data[column] = self.data[column].interpolate()
                
        print("Imputed values for any missing data")

In [15]:
#Assignment 9
class ClientManagerView:
    def __init__(self, controllerP):
        self.controller = controllerP
        
    def showInterface(self, output):
        self.output = output
        
        self.buildClientSearchInterface()
        self.buildClientCreationInterface()
        self.buildPortfolioListInterface()
        self.buildPortfolioDetailInterface()
        self.buildCIKSearchInterface()
        self.buildCIKReportInterface()
        
        self.interface = widgets.Tab()
        
        self.clientAccordion = widgets.Accordion([self.clientSearchInterface, self.clientCreationInterface])
        self.clientAccordion.set_title(0, "Find Client")
        self.clientAccordion.set_title(1, "New Client")
        
        self.portfolioAccordion = widgets.Accordion([self.portfolioListInterface, self.portfolioDetailInterface])
        self.portfolioAccordion.set_title(0, "Portfolio List")
        self.portfolioAccordion.set_title(1, "Portfolio Details")
        
        self.CIKAccordion = widgets.Accordion([self.CIKSearchInterface, self.CIKReportInterface])
        self.CIKAccordion.set_title(0, "Find Company CIK")
        self.CIKAccordion.set_title(1, "Generate CIK Report")
        
        self.interface.children = [self.clientAccordion, self.portfolioAccordion, self.CIKAccordion]
        self.interface.set_title(0, "Client Manager")
        self.interface.set_title(1, "Portfolio Manager")
        self.interface.set_title(2, "Company Report")
        
        display(self.interface, self.output)
    
    def buildClientSearchInterface(self):
        self.clientSearchHeader = widgets.HTML("<header class='sectionHeader'>Search for a client by last name:</header>")
        self.clientSearchBox = widgets.Text(placeholder="Enter a Last Name", description="Search:")
        self.clientSearchButton = widgets.Button(description="Search", tooltip="Search for a client by last name")
        
        self.clientSearchButton.on_click(self.controller.findByLastName)
        
        self.clientSearchInterface = widgets.VBox([self.clientSearchHeader, self.clientSearchBox, self.clientSearchButton])
        
    def buildClientSearchResults(self, clients):
        with self.output:
            self.clientSearchInterface.children = (self.clientSearchHeader, self.clientSearchBox, self.clientSearchButton)
            
            resultsHeader = widgets.HTML("<header class='sectionHeader'>The following clients matched your criteria:</header>")  #Assignment 8

            resultsBoxArray = [resultsHeader]

            for (clientId, firstName, lastName, phone) in clients:           
                selectButton = widgets.Button(description="Select")
                selectButton.clientId = clientId
                selectButton.on_click(self.controller.selectClientById)
                clientInfoLabel = widgets.Label(value=f"{firstName} {lastName} with phone {phone}")
                rowBox = widgets.HBox([selectButton, clientInfoLabel])

                resultsBoxArray.append(rowBox)

            resultsBox = widgets.VBox(resultsBoxArray)
            
            self.clientSearchInterface.children = self.clientSearchInterface.children + (resultsBox, )
        
    def buildClientCreationInterface(self):
        self.clientCreationHeader = widgets.HTML("<header class='sectionHeader'>Create a new client:</header>")
        self.clientFirstNameBox = widgets.Text(description="First Name:")
        self.clientLastNameBox = widgets.Text(description="Last Name:")
        self.clientPhoneBox = widgets.Text(description="Phone:")
        self.clientCreateButton = widgets.Button(description="Create")
        
        self.clientCreationInterface = widgets.VBox([self.clientCreationHeader, self.clientFirstNameBox, self.clientLastNameBox, self.clientPhoneBox, self.clientCreateButton])
        
    def buildPortfolioListInterface(self):
        self.portfolioListHeader = widgets.HTML("<header class='sectionHeader'>The client's portfolios:</header>")
        
        self.portfolioListInterface = widgets.VBox([self.portfolioListHeader])
        
    def buildPortfolioListResults(self, portfolios):
        with self.output:
            self.portfolioListInterface.children = (self.portfolioListHeader, )
            
            resultsBoxArray = []
            
            for (portfolioId, name) in portfolios:
                selectButton = widgets.Button(description="Select")
                selectButton.portfolioId = portfolioId
                selectButton.on_click(self.controller.selectPortfolioById)
                portfolioInfoLabel = widgets.Label(value=f"{name}")
                rowBox = widgets.HBox([selectButton, portfolioInfoLabel])
                
                resultsBoxArray.append(rowBox)
                
            resultsBox = widgets.VBox(resultsBoxArray)
            
            self.portfolioListInterface.children = self.portfolioListInterface.children + (resultsBox, )
            
            self.interface.selected_index = 1
        
    def buildPortfolioDetailInterface(self):
        self.portfolioDetailHeader = widgets.HTML("<header class='sectionHeader'>Portfolio details:</header>")
        
        self.portfolioDetailInterface = widgets.VBox([self.portfolioDetailHeader])

    def buildCIKSearchInterface(self):
        self.CIKSearchHeader = widgets.HTML("<header class='sectionHeader'>Search for a company's CIK number:</header>")
        self.CIKSearchBox = widgets.Text(placeholder="Enter a Company Name", description="Search:")
        self.CIKSearchButton = widgets.Button(description="Search", tooltip="Search for a company's CIK number")
        
        self.CIKSearchButton.on_click(self.controller.findCIKByCompanyName)
        
        self.CIKSearchInterface = widgets.VBox([self.CIKSearchHeader, self.CIKSearchBox, self.CIKSearchButton])
        
    def buildCIKReportInterface(self):
        self.CIKReportHeader = widgets.HTML("<header class='sectionHeader'>Financial Report</header>")
        self.CIKSelectBox = widgets.Text(placeholder="Enter a CIK Number", description="Search:")
        self.CIKSelectButton = widgets.Button(description="Get Report", tooltip="Build a report for the CIK number")
        
        self.CIKSelectButton.on_click(self.controller.getReportByCIK)
        
        self.CIKReportInterface = widgets.VBox([self.CIKReportHeader, self.CIKSelectBox, self.CIKSelectButton])
    
    def showMatchingCIKNumbers(self, results):
        with self.output:
            self.CIKSearchInterface.children = (self.CIKSearchHeader, self.CIKSearchBox, self.CIKSearchButton)
            
            html = f"""
                <pre>
                    {results}
                </pre>
            """
            
            self.CIKSearchInterface.children = self.CIKSearchInterface.children + (widgets.HTML(html), )
    
    def showCIKReport(self, df):
        with self.output:
            self.output.clear_output()
            display(df)
                
    def showSuccessMessage(self, message):
        with self.output:
            html = widgets.HTML(f"<div class='successMessage'>{message}</div>")
            display(html)
            time.sleep(1)
            self.output.clear_output()
                    
    def showWarningMessage(self, message):
        with self.output:
            html = widgets.HTML(f"<div class='warningMessage'>{message}</div>")
            display(html)
            time.sleep(1)
            self.output.clear_output()
            

In [16]:
dbConnector = MySQLConnector(hostP="mis2100example.mis.sbe.mtu.edu", databaseP="mis2100example_portfolio_db", userP="mis2100example_student", passwordP="BmKJtu-3RUp5")
dbConnector.connect()

manager = ClientManager(dbConnector)
#Assignment 9
view = ClientManagerView(manager) 
manager.setView(view)

2013 (HY000): Lost connection to MySQL server at 'reading initial communication packet', system error: 0


In [17]:
#Assignment 9
output = widgets.Output()
view.showInterface(output)

Output()

In [18]:
#data = GOOG.getStockHistory("2020-01-01", "2022-01-01")
data = pd.read_excel("METAStockData.xlsx")
cleaner = PandasStockDataCleaner(data)
cleaner.forceNumericDataTypes()
cleaner.identifyOutliers()
cleaner.calculateSkewnessAndKurtosis()
cleaner.showHistogramByColumn("Volume")
cleaner.imputeMissingData()

FileNotFoundError: [Errno 2] No such file or directory: 'METAStockData.xlsx'

In [ ]:
#Assignment 3
#create simple MockPurchase class
class MockPurchase:
    def __init__(self):
        self.quantity = 0
        self.price = 0

#create unit test suite for Stock object
class StockTestSuite(unittest.TestCase):
    #create a unit test
    def testGetPerceptChangeIsCorrectForPriceIncrease(self):
        #setup MockPurchase
        purchase1 = MockPurchase()
        purchase2 = MockPurchase() #Assignment 5
        
        #setup Stock
        stock = Stock("TST", "Test")
        
        #store MockPurcahse in the Stock object
        stock.purchases.append(purchase1) #Assignment 4
        stock.purchases[0].quantity = 5
        stock.purchases[0].price = 12.35
        
        stock.purchases.append(purchase2) #Assignment 5
        stock.purchases[1].quantity = 6
        stock.purchases[1].price = 11.73
        
        #Figure out what the true value should be
        #old purchase value of 5*12.35 + 6*11.73 = 132.13
        #new price of 13.75 results in new value of 11*13.75 = $151.25
        #(151.25 - 132.13) / 132.13 = 0.14471 * 100 = 14.47060
        
        percentChange = stock.getPercentChange(13.75)
        self.assertAlmostEqual(percentChange, 14.47060, 5)
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)